# Agents
### Example project: Your Personal Finance Expert
We'll finally combine llm tool-calling into a finance expert agent. It will be able to decide between tools (searching online vs. local documents). We'll also include memory for the agent so that it can retain conversation history.

In [32]:
# general imports
from dotenv import load_dotenv
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' # in case of vector store duplicate issues
load_dotenv()

# tools
from langchain_community.tools import TavilySearchResults
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_core.tools import tool

# langchain helpers
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub # for community prompts

# vectorstore (RAG)
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# agent imports
from langchain.agents import create_tool_calling_agent, AgentExecutor

# llm
base_url='http://localhost:11434'
llm = ChatOllama(model='llama3.2', base_url=base_url)
embeddings = OllamaEmbeddings(base_url=base_url, model='nomic-embed-text')

## Define Tools
### 1. Tavily (web search) tool

In [22]:
@tool
def tavily_search(query: str) -> str:
    """
    Search the web for realtime and latest news and information.
    For example, news, stock market, weather updates, etc.

    Args:
    query: The search query
    """
    search = TavilySearchResults(
    max_results=5,
    search_depth='advanced',
    include_answer=True,
    include_raw_content=True
    )
    return search.invoke(query)

### 2. Finance Pdfs Retriever (RAG finance pdfs) tool
Documents are already embedded and stored in vector database. Just need to load the vector store to retrieve these. Remember to use the **same embedding model for storage and retrieval**, otherwise results will NOT be accurate!

In [26]:
db_path = 'finance_books_vectorstore'
vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type='similarity', 
                                      search_kwargs={'k': 3})

In [27]:
# create function and wrap as tool; remember to provide a description
@tool
def finance_pdf_search(query: str) -> str:
    """
    Search for information about financial advice. 
    Use this tool for any questions about investment strategies, how to start investing, or financial advice.

    Args:
    query: The search query
    """
    return retriever.invoke(query)

## Agent Creation
Agent should: 
- Take input and decide what tool to use
- Agent will not execute the tool. This should be done by the AgentExecutor.

Use example prompt (very basic): https://smith.langchain.com/hub/hwchase17/openai-functions-agent \
Example more advanced prompt (text to SQL): https://smith.langchain.com/hub/miracle/anthropic_sql_query_agent

In [34]:
# getting the agent prompt from langchain hub. Good starting point, but can be fine-tuned.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Create agent; remember agent does NOT call the tools.

In [35]:
tools = [tavily_search, finance_pdf_search]
agent = create_tool_calling_agent(llm, tools, prompt)

Create agent executor to actually call the tools and return the information back to the agent.

In [37]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
question = "What advice do you have for a beginner first starting to invest?"
response = agent_executor.invoke({'input': question})




> Entering new AgentExecutor chain...

Invoking: `finance_pdf_search` with `{'query': 'beginner investment advice'}`


[Document(id='2a6c9e84-b107-4167-bdc3-409d3cd624dc', metadata={'producer': 'Acrobat Distiller 5.0 (Windows)', 'creator': 'QuarkXPress.: AdobePS 8.7.3 (301)', 'creationdate': '2005-08-01T23:34:28+08:00', 'source': 'finance_pdfs\\The Intelligent Investor - A Book of Practical Counsel 2003.pdf', 'file_path': 'finance_pdfs\\The Intelligent Investor - A Book of Practical Counsel 2003.pdf', 'total_pages': 643, 'format': 'PDF 1.6', 'title': 'The Intelligent Investor', 'author': 'Benjamin Graham, Jason Zweig and Warren E. Buffett', 'subject': '', 'keywords': '', 'moddate': '2008-01-05T15:44:22+00:00', 'trapped': '', 'modDate': 'D:20080105154422Z', 'creationDate': "D:20050801233428+08'00'", 'page': 291}, page_content='277\n6 If you have less than $100,000 to invest, you may not be able to find a\nfinancial adviser who will take your account. In that case, buy a diversified\nb

### This is the response from the agent!
Yellow refers to the documents returned as context, while green refers to the llm's inferences and output.

In [39]:
print(response['output'])

For a beginner starting to invest, here are some key pieces of advice:

1. **Start with a solid understanding**: Invest time in learning about personal finance, investing, and the stock market. This will help you make informed decisions.
2. **Diversify your portfolio**: Spread your investments across different asset classes, such as stocks, bonds, and real estate, to minimize risk.
3. **Keep costs low**: Look for low-cost index funds or ETFs, which can provide broad diversification at a lower cost than actively managed funds.
4. **Be patient and disciplined**: Investing is a long-term game. Avoid making emotional decisions based on short-term market fluctuations.
5. **Educate yourself**: Continuously learn about investing and personal finance to improve your skills and knowledge.
6. **Consider seeking professional advice**: If you're new to investing, consider consulting with a financial advisor or using online resources to get personalized advice.

Remember, investing is a journey, an

### Alternate query, this time not related to finance:

In [41]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Note: Set verbose to false to avoid having the giant chunk of text, if desired. Verbose can be used to debug and evaluate if returns are as desired.

In [42]:
question = "Current weather in Portland, Oregon"
response = agent_executor.invoke({'input': question})

In [43]:
print(response['output'])

The current weather in Portland, Oregon is mostly sunny with a temperature of 21.1°C (70°F). There is a gentle breeze coming from the northwest at 5.1 mph (8.3 km/h) and a pressure of 1017.0 mb. The humidity is relatively low at 28%, and there is no precipitation expected today. It feels like it's currently 21.1°C (70°F), which is the same as the temperature, due to the sunny conditions.


Final note: These agent calls can be traced through LangSmith for further details! 

Future work: Create chatbot interface for agent. Save conversation history.